# Dataset de supermercado para regras de associação

O conjunto de dados de comportamento do consumidor de supermercado consiste em 2.019.501 linhas e 12 colunas:
* **order_id** (Um número exclusivo para identificar o pedido)
* **user_id** - (Um número exclusivo para identificar o usuário)
* **order_number** (Número do pedido)
* **order_dow** (Dia da semana em que o pedido foi feito)
* **order_hour_of_day** (Hora do pedido)
* **dias_since_prior_order** (Histórico do pedido)
* **product_id** (Id do produto)
* **add_to_cart_order** (Número de itens adicionados ao carrinho)
* **reordenado** (se o novo pedido ocorreu)
* **departamental_id** (Número exclusivo alocado para cada departamento)
* **departament** (Nomes dos departamentos)
* **product_name** (Nome dos produtos)

**Fonte dos dados:**
O conjunto de dados consiste em mais de 2 milhões de registros de compras em um renomado supermercado Hunter :)

https://www.kaggle.com/datasets/hunter0007/ecommerce-dataset-for-predictive-marketing-2023/data

## Explicação do algorítimo Apriori
Suponha que temos a regra de associação
{Pão} →{Manteiga} com um suporte de **0.1**, uma confiança de **0.6** e um lift de **1.5**.

**Suporte** (Pão): 10% das transações contêm pão.

**Confiança** (Pão → Manteiga): 60% das transações que contêm pão também contêm manteiga.

**Lift** (Pão → Manteiga): A probabilidade de comprar pão e manteiga juntos é 1.5 vezes maior do que se fossem comprados independentemente.

Portanto, ao comprar pão, há uma probabilidade de 60% de que manteiga também seja comprada, e essa probabilidade é 1.5 vezes maior do que se pão e manteiga fossem comprados independentemente. Isso ajuda a identificar associações interessantes nos dados de transações.

In [124]:
#instalar a biblioteca apyori
#!pip install apyori

In [125]:
import pandas as pd
from apyori import apriori


In [126]:
#carga de dados
dados = pd.read_csv('ECommerce_consumer behaviour.csv')
dados.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,department_id,department,product_name
0,2425083,49125,1,2,18,NaN,17,1,0,13,pantry,baking ingredients
1,2425083,49125,1,2,18,NaN,91,2,0,16,dairy eggs,soy lactosefree
2,2425083,49125,1,2,18,NaN,36,3,0,16,dairy eggs,butter
3,2425083,49125,1,2,18,NaN,83,4,0,4,produce,fresh vegetables
4,2425083,49125,1,2,18,NaN,83,5,0,4,produce,fresh vegetables


Para esse projeto, iremos utilizar apenas as colunas **order_id** e **product_name**

In [127]:
#manter somente order_id e product_name
dados = dados[['order_id', 'product_name']]
dados.head()

,order_id,product_name
0,2425083,baking ingredients
1,2425083,soy lactosefree
2,2425083,butter
3,2425083,fresh vegetables
4,2425083,fresh vegetables


In [128]:
#shape do dataset
dados.shape

(2019501, 2)

In [129]:
#remove linhas com produtos 'missing'
dados = dados[dados['product_name'] != 'missing']

In [130]:
#descreve a quantidade de produtos
count_prod = dados.product_name.value_counts()
count_prod.describe()

count       133.000000
mean      15148.511278
std       30059.403362
min         279.000000
25%        2020.000000
50%        6321.000000
75%       17408.000000
max      226039.000000
Name: product_name, dtype: float64

In [131]:
#separar apenas uma amostra de dados (dataset muito grande)
dados_amostra = dados[:20000]


In [132]:
#media de produtos por transação
dados_amostra.groupby('order_id').count().mean()

product_name    9.985022
dtype: float64

In [133]:
#agrupar por order_id, e criar uma lista de produtos para cada ordem.

ordem_prod_agrupados = dados_amostra.groupby('order_id')['product_name'].apply(list).reset_index().drop(columns=['order_id'])
ordem_prod_agrupados = ordem_prod_agrupados['product_name'].values

ordem_prod_agrupados

array([list(['frozen meals', 'fresh dips tapenades', 'canned meals beans', 'chips pretzels', 'oils vinegars']),
       list(['pasta sauce', 'water seltzer sparkling water', 'chips pretzels', 'frozen produce', 'water seltzer sparkling water', 'fresh fruits']),
       list(['other creams cheeses', 'refrigerated', 'soup broth bouillon', 'yogurt', 'yogurt']),
       ..., list(['packaged cheese', 'pasta sauce']),
       list(['prepared soups salads', 'prepared soups salads', 'prepared soups salads']),
       list(['coffee', 'coffee', 'water seltzer sparkling water', 'fresh herbs', 'yogurt', 'dry pasta', 'packaged vegetables fruits', 'milk', 'water seltzer sparkling water', 'pasta sauce', 'asian foods', 'fresh herbs', 'eggs'])],
      dtype=object)

In [134]:
#para cada transação, considerar apenas os 20 primeiros itens, sendo que nas transações
# (somente quando houver muitos produtos no carrinho, mas não é obrigatório)

transactions = []

for x in range(len(ordem_prod_agrupados)):
    linha = ordem_prod_agrupados[x][:20]
    transactions.append(linha)
transactions

[['frozen meals',
  'fresh dips tapenades',
  'canned meals beans',
  'chips pretzels',
  'oils vinegars'],
 ['pasta sauce',
  'water seltzer sparkling water',
  'chips pretzels',
  'frozen produce',
  'water seltzer sparkling water',
  'fresh fruits'],
 ['other creams cheeses',
  'refrigerated',
  'soup broth bouillon',
  'yogurt',
  'yogurt'],
 ['water seltzer sparkling water',
  'refrigerated',
  'refrigerated',
  'coffee',
  'yogurt',
  'tea'],
 ['butter',
  'chips pretzels',
  'breakfast bakery',
  'food storage',
  'fresh dips tapenades',
  'trail mix snack mix',
  'soft drinks',
  'crackers'],
 ['cream',
  'milk',
  'milk',
  'packaged vegetables fruits',
  'fresh vegetables',
  'packaged cheese',
  'packaged cheese',
  'cereal',
  'fresh fruits',
  'fresh vegetables',
  'spreads',
  'fresh fruits'],
 ['ice cream ice', 'fresh dips tapenades', 'fresh fruits'],
 ['frozen appetizers sides',
  'crackers',
  'spreads',
  'popcorn jerky',
  'lunch meat',
  'soup broth bouillon',
  'fr

## Parâmetros do apriori
* **transactions:** lista com listas de items. Ex: [[banana, maçã, peixe], [maça, arroz, frutas], [arroz, banana, leite]

* **min_support:** percentual minimo que produto aparece nas transações (0.01 = 1%)

* **min_confidence:** percentual mínimo que outro produto comprado com o produto principal aparece nas mesmas transações (0.1 = 10%)

* **min_lift:** Usado para medir o quão mais provável é a ocorrência conjunta do produto A com produto B. Maior que 1 indica uma associação positiva, enquanto um lift menor que 1 indica uma associação negativa.

* **min_length:** minimo de combinação de produtos (se for 1 o produto será comparado a ele mesmo)

* **max_length:** máximo de combinação de produtos


In [171]:
# execução do apriori para descobrir combinações
resultado = list(apriori(
    transactions = transactions,
    min_support = 0.01, # percentual minimo que produto aparece nas transações (0.01 = 1%)
    min_confidence = 0.1, # percentual mínimo que outro produto comprado com o produto principal aparece nas mesmas transações (0.1 = 10%)
    min_lift = 1.2, # Usado para medir o quão mais provável é a ocorrência conjunta do produto A com produto B.
                    # Maior que 1 indica uma associação positiva, enquanto um lift menor que 1 indica uma associação negativa.
    min_length = 2, # minimo de combinação de produtos (se for 1 o produto será comparado a ele mesmo)
    max_length = 3 # máximo de combinação de produtos
    ))


In [172]:
#importa o resultado do apriori para um dataframe
resultado_df = pd.DataFrame()

for x in range (len(resultado)):
    nova_linha = {
        'Items Base' : ', '.join(list(resultado[x][2][0][0])),
        'Items Adicional' : ', '.join(list(resultado[x][2][0][1])),
        'Suporte' : resultado[x][1],
        'Confiança': resultado[x][2][0][2],
        'Lift' : resultado[x][2][0][3]
    }

    resultado_df = pd.concat([resultado_df, pd.DataFrame([nova_linha])], ignore_index=True)

#exibe o dataframe ordenando os maiores lifts
resultado_df.sort_values(by=['Lift'], ascending=False)


,Items Base,Items Adicional,Suporte,Confiança,Lift
694,dry pasta,"pasta sauce, fresh fruits",0.011982,0.208696,5.291359
172,dry pasta,pasta sauce,0.017474,0.304348,5.210331
698,dry pasta,"fresh vegetables, pasta sauce",0.010484,0.182609,5.151623
528,canned jarred vegetables,"fresh vegetables, soup broth bouillon",0.013480,0.198529,4.230366
940,fresh herbs,"fresh vegetables, oils vinegars",0.011483,0.140244,4.131008
...,...,...,...,...,...
31,bread,frozen produce,0.020969,0.132075,1.202487
506,"butter, fresh vegetables",yogurt,0.011483,0.310811,1.201842
151,crackers,packaged vegetables fruits,0.042936,0.403756,1.201669
177,eggs,fresh fruits,0.089366,0.660517,1.201648


## Conclusão
Na coluna Lift representa a probalidade maior de itens associados saírem juntos do que separado. Ex: *{dry pasta, pasta sauce, fresh fruits}* tem **5 vezes** mais chances de saírem juntos, do que se estiverem vendidos separadamente.
